In [ ]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import random
import torchvision.transforms as transforms
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from google.colab import drive


import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
# Training function.
def train(epoch, model, loader, criterion, optimizer, device='cpu'):
    l = 0
    for data in tqdm(loader, desc=f'Epoch {epoch+1:03d}'):
        x = data[0].to(device)
        y = data[1].squeeze().to(device)
        out = model(x)
        loss = criterion(out, y)
        l += loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return l

# Test function.
def test(model, loader, criterion, device='cpu'):
    l = 0
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in loader:
            x = data[0].to(device)
            y = data[1].squeeze().to(device)
            out = model(x)
            l += criterion(out, y)
            _, pred = torch.max(out.data, 1)
            total += y.size(0)
            correct += (pred == y).sum().item()
            y_true += y.tolist()
            y_pred += pred.tolist()
    return l, correct / total, y_true, y_pred

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms

# Mounta il drive di Google
drive.mount('/content/drive')

# Definisci le cartelle delle immagini
train_dir = '/content/drive/MyDrive/UNI/Magistrale/Applicazioni_IA/Progetto/Images/train'
test_dir = '/content/drive/MyDrive/UNI/Magistrale/Applicazioni_IA/Progetto/Images/test'

# Definisci le etichette
etichette = ['N', 'A', 'V']

# Crea un dizionario per mappare le etichette ai valori numerici
etichette_map = {etichetta: i for i, etichetta in enumerate(etichette)}

# Funzione per caricare le immagini e le loro etichette
class ImmaginiDataset(torch.utils.data.Dataset):
    def __init__(self, dir, transform=None):
        self.dir = dir
        self.transform = transform
        self.images = []
        for etichetta in etichette:
            path = os.path.join(dir, etichetta)
            for file in os.listdir(path):
                self.images.append((os.path.join(path, file), etichetta))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        path, etichetta = self.images[index]
        #immagine = torchvision.io.read_image(path, mode=torchvision.io.ImageReadMode.LZW)
        immagine = Image.open(path).convert('L') # Converti in bianco e nero
        if self.transform:
            immagine = self.transform(immagine)
        return immagine, etichette_map[etichetta]

# Crea il dataset per l'addestramento
train_dataset = ImmaginiDataset(train_dir)

# Applica le trasformazioni alle immagini
train_transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset.transform = train_transformations

# Crea il dataset per il test
test_dataset = ImmaginiDataset(test_dir)
test_dataset.transform = train_transformations

# Crea i dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


In [ ]:
# Device configuration.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# Hyper-parameters.
num_classes = len(etichette)
num_epochs = 5
learning_rate = 0.001

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3)
        )
        self.fc = nn.Linear(23 * 23 * 16, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

# Create the model.
model = ConvNet(num_classes).to(device)

In [ ]:
# Loss and optimizer.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training and test.
for epoch in range(num_epochs):
    train_loss = train(epoch, model, train_loader, criterion, optimizer, device)
    test_loss, test_acc, y_true, y_pred = test(model, test_loader, criterion, device)
    print(f'Epoch {epoch+1:03d}: training loss {train_loss:.4f}, test loss {test_loss:.4f}, test acc {test_acc:.4f}')

In [ ]:
print(classification_report(y_true, y_pred, zero_division=0))